학번: 202120634 / 짝수로 끝나는 학생은 wine_review.csv를 분석

In [49]:
import pandas as pd

df = pd.read_csv('./wine_review.csv')
df.head()

,id,asins,brand,categories,dateAdded,dateUpdated,descriptions,dimension,ean,flavors,...,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sizes,sourceURLs,upc,weight
0,AV13ClKCGV-KLJ3akN68,NaN,Gallo,"Food & Beverage,Beverages,Wine, Beer & Liquor,...",2017-07-24T23:59:11Z,2018-01-10T18:06:28Z,NaN,1.0 in x 1.0 in x 1.0 in,NaN,NaN,...,https://redsky.target.com/groot-domain-api/v1/...,This a fantastic white wine for any occasion!,My Favorite White Wine,NaN,NaN,Bjh,NaN,http://redsky.target.com/v1/plp/search?kwr=y&c...,4.9213E+11,1.0 lbs
1,AV13CsvW-jtxr-f38AQO,NaN,Fresh Craft Co.,"Food & Beverage,Beverages,Wine, Beer & Liquor,...",2017-07-24T23:59:42Z,2018-01-10T05:38:33Z,"[{""dateSeen"":[""2017-12-21T05:43:00.000Z"",""2017...",4.25 in x 4.25 in x 5.25 in,NaN,NaN,...,https://redsky.target.com/groot-domain-api/v1/...,"Tart, not sweet...very refreshing and delicious!",Yum!!,NaN,NaN,Wino,NaN,http://redsky.target.com/v1/plp/search?kwr=y&c...,83120003441,2.45 lbs
2,AV13CVI_glJLPUi8O7Po,NaN,1000 Stories,"Food & Beverage,Beverages,Wine, Beer & Liquor,...",2017-07-24T23:58:05Z,2018-01-10T05:38:31Z,NaN,3.3 in x 3.3 in x 11.79 in,NaN,NaN,...,https://redsky.target.com/groot-domain-api/v1/...,I was given this wine so it was a delightful s...,A New Favorite!,NaN,NaN,Bama Mom,NaN,http://redsky.target.com/v1/plp/search?kwr=y&c...,82896001453,3.09 lbs
3,AV13CVI_glJLPUi8O7Po,NaN,1000 Stories,"Food & Beverage,Beverages,Wine, Beer & Liquor,...",2017-07-24T23:58:05Z,2018-01-10T05:38:31Z,NaN,3.3 in x 3.3 in x 11.79 in,NaN,NaN,...,https://redsky.target.com/groot-domain-api/v1/...,This is a phenomenal wine and my new favorite ...,"Bold, Flavorful, Aromatic, Delicious",NaN,NaN,Av Dub,NaN,http://redsky.target.com/v1/plp/search?kwr=y&c...,82896001453,3.09 lbs
4,AV13CYL4-jtxr-f37_-t,NaN,Wine Cube153,"Food & Beverage,Beverages,Wine, Beer & Liquor,...",2017-07-24T23:58:18Z,2018-01-10T18:06:29Z,"[{""dateSeen"":[""2017-12-21T05:43:00.000Z"",""2017...",1.0 in x 1.0 in x 1.0 in,NaN,NaN,...,https://redsky.target.com/groot-domain-api/v1/...,4 750ml bottles for the price of two With way ...,"Yum! Plus, Environmentally Friendly!",Overland Park,NaN,Chelseamay,NaN,https://redsky.target.com/groot-domain-api/v1/...,85200600465,1.0 lbs


데이터 정리 (노이즈 제거, 결측값 처리 등)

In [50]:
# 필요한 라이브러리 설치 및 임포트
%pip install nltk
import nltk
import pandas as pd
import re

# NLTK 데이터 다운로드
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# 데이터 불러오기 
df = pd.read_csv('wine_review.csv')

# 결측치 처리: 'reviews.text'와 'reviews.rating' 열의 결측값을 빈 문자열로 채움
df['reviews.text'].fillna('', inplace=True)
df['reviews.rating'].fillna(0, inplace=True)

# 출력 확인
print(df[['reviews.text']].head())

Note: you may need to restart the kernel to use updated packages.
                                        reviews.text
0      This a fantastic white wine for any occasion!
1   Tart, not sweet...very refreshing and delicious!
2  I was given this wine so it was a delightful s...
3  This is a phenomenal wine and my new favorite ...
4  4 750ml bottles for the price of two With way ...


[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/tmp/ipykernel_19949/4083939650.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['reviews.text'].fillna('', inplace=Tr

텍스트 토큰화 및 불용어 제거

In [51]:
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# 데이터 전처리 준비
stop_words = set(stopwords.words('english'))

# 텍스트 토큰화 및 불용어 제거 함수 정의
def tokenize_and_remove_stopwords(text):
    
    # 특수문자 제거
    text = re.sub(r'[^\w\s]', '', text)
    # 숫자 제거
    text = re.sub(r'\d+', '', text)
    # 불필요한 공백 제거
    text = ' '.join(text.split())
    # 토큰화
    tokens = word_tokenize(text)
    # 불용어 제거
    tokens = [word.lower() for word in tokens if word.lower() not in stop_words]
    return tokens

# 토큰화 및 불용어 제거 적용
df['tokenized_text'] = df['reviews.text'].apply(tokenize_and_remove_stopwords)

# 출력 확인
print(df[['reviews.text', 'tokenized_text']])

                                           reviews.text  \
0         This a fantastic white wine for any occasion!   
1      Tart, not sweet...very refreshing and delicious!   
2     I was given this wine so it was a delightful s...   
3     This is a phenomenal wine and my new favorite ...   
4     4 750ml bottles for the price of two With way ...   
...                                                 ...   
2885  I only like a sweet wine. I was skeptical on o...   
2886  I ordered a 3 bottle set of both the Meritage ...   
2887  I ordered the White Zinfadel in the 3 bottle s...   
2888  This was my first purchase of these wines. Tri...   
2889  I don't know why the negative reviews said the...   

                                         tokenized_text  
0                    [fantastic, white, wine, occasion]  
1              [tart, sweetvery, refreshing, delicious]  
2     [given, wine, delightful, surprise, find, flav...  
3                [phenomenal, wine, new, favorite, red]  
4

텍스트 정규화 (스테밍 또는 표제어 추출)

In [52]:
from nltk.stem import PorterStemmer, WordNetLemmatizer

# 스테밍과 표제어 추출 준비
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# 텍스트 정규화 함수 정의 (스테밍 및 표제어 추출)
def normalize_text(tokens):
    # 스테밍 및 표제어 추출
    normalized_tokens = [lemmatizer.lemmatize(stemmer.stem(token)) for token in tokens]
    return ' '.join(normalized_tokens)

# 정규화 적용
df['processed_text'] = df['tokenized_text'].apply(normalize_text)

# 출력 확인
print(df[['reviews.text', 'processed_text', 'reviews.rating']].head())

# 전처리된 텍스트와 점수를 포함하는 파일로 저장
df[['reviews.text', 'processed_text', 'reviews.rating']].to_csv('cleaned_wine_review.csv', index=False)

                                        reviews.text  \
0      This a fantastic white wine for any occasion!   
1   Tart, not sweet...very refreshing and delicious!   
2  I was given this wine so it was a delightful s...   
3  This is a phenomenal wine and my new favorite ...   
4  4 750ml bottles for the price of two With way ...   

                                      processed_text  reviews.rating  
0                           fantast white wine occas             5.0  
1                      tart sweetveri refresh delici             5.0  
2  given wine delight surpris find flavor delici ...             5.0  
3                      phenomen wine new favorit red             5.0  
4  ml bottl price two way le packag ye plea nervo...             5.0  


VADER 감정 분석

In [53]:
%pip install vaderSentiment
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# 데이터 불러오기
df = pd.read_csv('cleaned_wine_review.csv')

# NaN 값 처리: 빈 문자열로 대체
df['processed_text'].fillna('', inplace=True)

# VADER 초기화
analyzer = SentimentIntensityAnalyzer()

# 감정 점수 계산 함수 정의
def analyze_sentiment(text):
    sentiment = analyzer.polarity_scores(text)
    return sentiment['compound']

# 'processed_text' 열의 각 텍스트에 대해 감정 점수 계산
df['sentiment_score'] = df['processed_text'].apply(analyze_sentiment)

# 결과 확인
print(df[['processed_text', 'sentiment_score']])

Note: you may need to restart the kernel to use updated packages.


/tmp/ipykernel_19949/968434302.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['processed_text'].fillna('', inplace=True)


                                         processed_text  sentiment_score
0                              fantast white wine occas           0.0000
1                         tart sweetveri refresh delici           0.0000
2     given wine delight surpris find flavor delici ...           0.5994
3                         phenomen wine new favorit red           0.0000
4     ml bottl price two way le packag ye plea nervo...           0.6694
...                                                 ...              ...
2885  like sweet wine skeptic order without tast fir...           0.8325
2886  order bottl set meritag moscata wine friend hu...           0.9545
2887  order white zinfadel bottl select arriv friday...           0.9118
2888  first purchas wine tri moscato like sweet wine...           0.8442
2889  dont know neg review said wine tast horribl or...           0.0000

[2890 rows x 2 columns]
